# Store Sales - Time Series Forecasting

### Use machine learning to predict grocery sales

Kaggle link: [Store Sales - Time Series Forecasting](https://www.kaggle.com/competitions/store-sales-time-series-forecasting)

### Context
Forecasts aren’t just for meteorologists. Governments forecast economic growth. Scientists attempt to predict the future population. And businesses forecast product demand—a common task of professional data scientists. Forecasts are especially relevant to brick-and-mortar grocery stores, which must dance delicately with how much inventory to buy. Predict a little over, and grocers are stuck with overstocked, perishable goods. Guess a little under, and popular items quickly sell out, leading to lost revenue and upset customers. More accurate forecasting, thanks to machine learning, could help ensure retailers please customers by having just enough of the right products at the right time.

Current subjective forecasting methods for retail have little data to back them up and are unlikely to be automated. The problem becomes even more complex as retailers add new locations with unique needs, new products, ever-transitioning seasonal tastes, and unpredictable product marketing.

### Potential Impact
If successful, you'll have flexed some new skills in a real world example. For grocery stores, more accurate forecasting can decrease food waste related to overstocking and improve customer satisfaction. The results of this ongoing competition, over time, might even ensure your local store has exactly what you need the next time you shop.

### Dataset Description
In this competition, you will predict sales for the thousands of product families sold at Favorita stores located in Ecuador. The training data includes dates, store and product information, whether that item was being promoted, as well as the sales numbers. Additional files include supplementary information that may be useful in building your models.

### File Descriptions and Data Field Information

**train.csv**
The training data, comprising time series of features store_nbr, family, and onpromotion as well as the target sales.
**store_nbr** identifies the store at which the products are sold.
**family** identifies the type of product sold.
**sales** gives the total sales for a product family at a particular store at a given date. Fractional values are possible since products can be sold in fractional units (1.5 kg of cheese, for instance, as opposed to 1 bag of chips).
**onpromotion** gives the total number of items in a product family that were being promoted at a store at a given date.

**test.csv**
The test data, having the same features as the training data. You will predict the target sales for the dates in this file.
The dates in the test data are for the 15 days after the last date in the training data.

**sample_submission.csv**
A sample submission file in the correct format.

**stores.csv**
Store metadata, including city, state, type, and cluster.
cluster is a grouping of similar stores.

**oil.csv**
Daily oil price. Includes values during both the train and test data timeframes. (Ecuador is an oil-dependent country and it's economical health is highly vulnerable to shocks in oil prices.)

**holidays_events.csv**
* Holidays and Events, with metadata
* NOTE: Pay special attention to the transferred column. A holiday that is transferred officially falls on that calendar day, but was moved to another date by the government. A transferred day is more like a normal day than a holiday. To find the day that it was actually celebrated, look for the corresponding row where type is Transfer. For example, the holiday Independencia de Guayaquil was transferred from 2012-10-09 to 2012-10-12, which means it was celebrated on 2012-10-12. Days that are type Bridge are extra days that are added to a holiday (e.g., to extend the break across a long weekend). These are frequently made up by the type Work Day which is a day not normally scheduled for work (e.g., Saturday) that is meant to payback the Bridge.
* Additional holidays are days added a regular calendar holiday, for example, as typically happens around Christmas (making Christmas Eve a holiday).

### Additional Notes

* Wages in the public sector are paid every two weeks on the 15 th and on the last day of the month. Supermarket sales could be affected by this.
* A magnitude 7.8 earthquake struck Ecuador on April 16, 2016. People rallied in relief efforts donating water and other first need products which greatly affected supermarket sales for several weeks after the earthquake.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import *
from sklearn.linear_model import *
from math import *
from sklearn.ensemble import *
from sklearn.feature_selection import *
from sklearn.feature_extraction import *
from sklearn.naive_bayes import *
from sklearn.discriminant_analysis import *
from sklearn.preprocessing import *
from sklearn.metrics import *
from sklearn.neighbors import *
from sklearn.cluster import *
from sklearn.svm import *
import warnings
warnings.filterwarnings("ignore")

In [2]:
df_1 = pd.read_csv("train1.csv")
df_1

,id,date,store_nbr,family,sales,onpromotion
0,0,1/1/13,1,AUTOMOTIVE,0.0,0
1,1,1/1/13,1,BABY CARE,0.0,0
2,2,1/1/13,1,BEAUTY,0.0,0
3,3,1/1/13,1,BEVERAGES,0.0,0
4,4,1/1/13,1,BOOKS,0.0,0
...,...,...,...,...,...,...
262138,262138,5/28/13,14,LADIESWEAR,0.0,0
262139,262139,5/28/13,14,LAWN AND GARDEN,0.0,0
262140,262140,5/28/13,14,LINGERIE,4.0,0
262141,262141,5/28/13,14,"LIQUOR,WINE,BEER",13.0,0


In [3]:
df_2 = pd.read_csv("train2.csv")
df_2

,id,date,store_nbr,family,sales,onpromotion
0,262143,5/28/13,14,MEATS,310.173000,0
1,262144,5/28/13,14,PERSONAL CARE,227.000000,0
2,262145,5/28/13,14,PET SUPPLIES,0.000000,0
3,262146,5/28/13,14,PLAYERS AND ELECTRONICS,0.000000,0
4,262147,5/28/13,14,POULTRY,47.809002,0
...,...,...,...,...,...,...
262138,524281,10/22/13,2,EGGS,153.000000,0
262139,524282,10/22/13,2,FROZEN FOODS,86.259000,0
262140,524283,10/22/13,2,GROCERY I,2538.000000,0
262141,524284,10/22/13,2,GROCERY II,22.000000,0


In [4]:
df_3 = pd.read_csv("train3.csv")
df_3

,id,date,store_nbr,family,sales,onpromotion
0,524286.0,10/22/13,2.0,HOME AND KITCHEN I,0.0,0.0
1,524287.0,10/22/13,2.0,HOME AND KITCHEN II,0.0,0.0
2,524288.0,10/22/13,2.0,HOME APPLIANCES,0.0,0.0
3,524289.0,10/22/13,2.0,HOME CARE,0.0,0.0
4,524290.0,10/22/13,2.0,LADIESWEAR,0.0,0.0
...,...,...,...,...,...,...
1048570,NaN,NaN,NaN,NaN,NaN,NaN
1048571,NaN,NaN,NaN,NaN,NaN,NaN
1048572,NaN,NaN,NaN,NaN,NaN,NaN
1048573,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df_4 = pd.read_csv("train4.csv")
df_4

,id,date,store_nbr,family,sales,onpromotion
0,786429.0,3/19/14,25.0,CELEBRATION,8.000,0.0
1,786430.0,3/19/14,25.0,CLEANING,485.000,0.0
2,786431.0,3/19/14,25.0,DAIRY,588.000,0.0
3,786432.0,3/19/14,25.0,DELI,116.357,0.0
4,786433.0,3/19/14,25.0,EGGS,119.000,0.0
...,...,...,...,...,...,...
1048570,NaN,NaN,NaN,NaN,NaN,NaN
1048571,NaN,NaN,NaN,NaN,NaN,NaN
1048572,NaN,NaN,NaN,NaN,NaN,NaN
1048573,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
a = df_1.append(df_2)
b = df_3.append(df_4)
df = a.append(b)
df.tail()

,id,date,store_nbr,family,sales,onpromotion
262141,1048570,8/13/14,3,POULTRY,974.098,1
262142,1048571,8/13/14,3,PREPARED FOODS,324.293,0
262143,1048572,8/13/14,3,PRODUCE,10.000,1
262144,1048573,8/13/14,3,SCHOOL AND OFFICE SUPPLIES,0.000,0
262145,1048574,8/13/14,3,SEAFOOD,69.116,0
